# Pre-Procesamiento

El objetivo de esta sección es realizar el pre-procesamiento de la información de nuestro análisis, a fin de aplicar distintas técnicas para la depuración y mejora en la calidad de información que trataremos.

Realizando una breve exploración, observamos que en los mensajes recibidos y que estaremos tratando contienen además del texto común, caracteres tipo emoticones, faltas de ortografía, abreviación de palabras y demás.

Aplicaremos:

*   BoW
*   Tf-Idf
*   PCA



In [3]:
!pip install transformers pandas scikit-learn accelerate emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 35.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-many

In [4]:
import torch
import pandas as pd
import numpy as np
import emoji
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


In [5]:
# Verificar si hay una GPU disponible y establecer el dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

# Imprimir información sobre los dispositivos CUDA
print(torch.cuda.is_available())
print(torch.cuda.device_count())
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))


Usando dispositivo: cuda
True
1
Tesla T4


In [6]:
# ejecutar luego de subir archivo excel o tener vinculado Google Drive
import os
import pandas as pd
from google.colab import drive

if os.path.isfile("/content/mensajes_trabajo_final.xlsx"):
  data = pd.read_excel("/content/mensajes_trabajo_final.xlsx")
else:
  # Montar Google Drive
  drive.mount('/content/drive')

  # Cambia el path al archivo que hayas subido a tu Google Drive
  file_path = '/content/drive/MyDrive/UCU-NLP/mensajes_trabajo_final_limpios.xlsx'

  # Cargar el archivo Excel
  data = pd.read_excel(file_path)

print("Datos cargados")

Mounted at /content/drive
Datos cargados


In [7]:
# Seleccionar las columnas relevantes
data = data[['enviado','recibido', 'clase']]

# Mostrar las primeras filas del archivo
print(data.head())

                                             enviado  \
0   ¡Limpiar la casa, subir escaleras, trabajar e...   
1   ¡Limpiar la casa, subir escaleras, trabajar e...   
2   ¡Limpiar la casa, subir escaleras, trabajar e...   
3   ¡Limpiar la casa, subir escaleras, trabajar e...   
4   ¡Limpiar la casa, subir escaleras, trabajar e...   

                                            recibido  \
0      ::pulgar_arriba:::manos formando un corazón::   
1                                           Hola....   
2       ::manos clamando::::manos clamando:: gracias   
3                                            Gracias   
4  Hola .. si suele..cuando me...desvelo..o de ta...   

                            clase  
0                  agradecimiento  
1                          saludo  
2                  agradecimiento  
3                  agradecimiento  
4  respuesta a consejo o pregunta  


**Análisis BOW**

In [8]:
# unir enviado con recibido
data['recibido_original'] = data['recibido']
data['recibido'] = data['enviado'] + " | " + data['recibido'].astype(str)

In [9]:
# agrupar clases

data.loc[data['clase'] == 'agradecimiento', 'clase'] = 'agradecimiento o saludo'
data.loc[data['clase'] == 'saludo', 'clase'] = 'agradecimiento o saludo'
data.loc[data['clase'] == 'medicación', 'clase'] = 'medicación o turno'
data.loc[data['clase'] == 'turno', 'clase'] = 'medicación o turno'
data.loc[data['clase'] == 'queja', 'clase'] = 'otros'
data.loc[data['clase'] == 'pregunta', 'clase'] = 'otros'
data.loc[data['clase'] == 'respuesta a consejo o pregunta', 'clase'] = 'respuesta'

In [10]:
# Convertir etiquetas de clase a representación numérica (suponiendo que son cadenas)
data['clase'] = data['clase'].astype('category')
data['clases_cat'] = data['clase'].cat.codes

In [11]:
# Diccionario de traducciones de emojis de inglés a español
emoji_translations = {
    ":grinning_face:": ":cara_sonriendo:",
    ":grinning_face_with_big_eyes:": ":cara_sonriendo_con_ojos_grandes:",
    ":grinning_face_with_smiling_eyes:": ":cara_sonriendo_con_ojos_sonrientes:",
    ":beaming_face_with_smiling_eyes:": ":cara_radiante_con_ojos_sonrientes:",
    ":grinning_squinting_face:": ":cara_sonriendo_con_ojos_cerrados:",
    ":grinning_face_with_sweat:": ":cara_sonriendo_con_sudor:",
    ":rolling_on_the_floor_laughing:": ":riendo_en_el_suelo:",
    ":face_with_tears_of_joy:": ":cara_con_lágrimas_de_alegría:",
    ":slightly_smiling_face:": ":cara_ligeramente_sonriente:",
    ":upside_down_face:": ":cara_al_revés:",
    ":winking_face:": ":cara_guiñando:",
    ":smiling_face_with_smiling_eyes:": ":cara_sonriendo_con_ojos_sonrientes:",
    ":smiling_face_with_halo:": ":cara_sonriendo_con_auréola:",
    ":smiling_face_with_hearts:": ":cara_sonriendo_con_corazones:",
    ":smiling_face_with_heart_eyes:": ":cara_sonriendo_con_ojos_de_corazón:",
    ":star_struck:": ":cara_deslumbrada:",
    ":face_blowing_a_kiss:": ":cara_lanzando_un_beso:",
    ":kissing_face:": ":cara_besando:",
    ":smiling_face:": ":cara_sonriente:",
    ":kissing_face_with_closed_eyes:": ":cara_besando_con_ojos_cerrados:",
    ":kissing_face_with_smiling_eyes:": ":cara_besando_con_ojos_sonrientes:",
    ":face_savoring_food:": ":cara_saboreando_comida:",
    ":face_with_tongue:": ":cara_con_lengua:",
    ":winking_face_with_tongue:": ":cara_guiñando_con_lengua:",
    ":zany_face:": ":cara_loca:",
    ":squinting_face_with_tongue:": ":cara_entrecerrada_con_lengua:",
    ":money_mouth_face:": ":cara_con_boca_de_dinero:",
    ":hugging_face:": ":cara_abrazando:",
    ":face_with_hand_over_mouth:": ":cara_con_mano_sobre_la_boca:",
    ":shushing_face:": ":cara_pidiendo_silencio:",
    ":thinking_face:": ":cara_pensando:",
    ":zipper_mouth_face:": ":cara_con_cierre_en_la_boca:",
    ":face_with_raised_eyebrow:": ":cara_con_ceja_levantada:",
    ":neutral_face:": ":cara_neutra:",
    ":expressionless_face:": ":cara_sin_expresión:",
    ":face_without_mouth:": ":cara_sin_boca:",
    ":smirking_face:": ":cara_sonriente:",
    ":unamused_face:": ":cara_no_divertida:",
    ":face_with_rolling_eyes:": ":cara_con_ojos_en_blanco:",
    ":grimacing_face:": ":cara_haciendo_mueca:",
    ":lying_face:": ":cara_mintiendo:",
    ":relieved_face:": ":cara_aliviada:",
    ":pensive_face:": ":cara_pensativa:",
    ":sleepy_face:": ":cara_somnolienta:",
    ":drooling_face:": ":cara_babeando:",
    ":sleeping_face:": ":cara_durmiendo:",
    ":face_with_medical_mask:": ":cara_con_mascarilla:",
    ":face_with_thermometer:": ":cara_con_termómetro:",
    ":face_with_head_bandage:": ":cara_con_venda_en_la_cabeza:",
    ":nauseated_face:": ":cara_nauseada:",
    ":face_vomiting:": ":cara_vomitando:",
    ":sneezing_face:": ":cara_estornudando:",
    ":hot_face:": ":cara_caliente:",
    ":cold_face:": ":cara_fría:",
    ":woozy_face:": ":cara_mareada:",
    ":dizzy_face:": ":cara_mareada:",
    ":exploding_head:": ":cabeza_explosiva:",
    ":cowboy_hat_face:": ":cara_con_sombrero_de_vaquero:",
    ":partying_face:": ":cara_festiva:",
    ":smiling_face_with_sunglasses:": ":cara_sonriendo_con_gafas_de_sol:",
    ":nerd_face:": ":cara_de_nerd:",
    ":face_with_monocle:": ":cara_con_monóculo:",
    ":confused_face:": ":cara_confundida:",
    ":worried_face:": ":cara_preocupada:",
    ":slightly_frowning_face:": ":cara_ligeramente_fruncida:",
    ":frowning_face:": ":cara_fruncida:",
    ":face_with_open_mouth:": ":cara_con_boca_abierta:",
    ":hushed_face:": ":cara_silenciosa:",
    ":astonished_face:": ":cara_asombrada:",
    ":flushed_face:": ":cara_sonrojada:",
    ":pleading_face:": ":cara_suplicante:",
    ":frowning_face_with_open_mouth:": ":cara_fruncida_con_boca_abierta:",
    ":anguished_face:": ":cara_angustiada:",
    ":fearful_face:": ":cara_temorosa:",
    ":cold_sweat:": ":sudor_frío:",
    ":disappointed_but_relieved_face:": ":cara_decepcionada_pero_aliviada:",
    ":crying_face:": ":cara_llorando:",
    ":loudly_crying_face:": ":cara_llorando_fuertemente:",
    ":face_screaming_in_fear:": ":cara_gritando_de_miedo:",
    ":confounded_face:": ":cara_confundida:",
    ":persevering_face:": ":cara_perseverante:",
    ":disappointed_face:": ":cara_decepcionada:",
    ":sweat:": ":sudor:",
    ":weary_face:": ":cara_cansada:",
    ":tired_face:": ":cara_fatigada:",
    ":yawning_face:": ":cara_bostezando:",
    ":face_with_steam_from_nose:": ":cara_con_vapor_saliendo_de_la_nariz:",
    ":pouting_face:": ":cara_haciendo_mueca:",
    ":angry_face:": ":cara_enojada:",
    ":face_with_symbols_on_mouth:": ":cara_con_símbolos_en_la_boca:",
    ":smiling_face_with_horns:": ":cara_sonriendo_con_cuernos:",
    ":angry_face_with_horns:": ":cara_enojada_con_cuernos:",
    ":skull:": ":calavera:",
    ":skull_and_crossbones:": ":calavera_y_huesos_cruzados:",
    ":pile_of_poo:": ":montón_de_caca:",
    ":clown_face:": ":cara_de_payaso:",
    ":ogre:": ":ogro:",
    ":goblin:": ":duende:",
    ":ghost:": ":fantasma:",
    ":alien:": ":extraterrestre:",
    ":alien_monster:": ":monstruo_alienígena:",
    ":robot:": ":robot:",
    ":grinning_cat:": ":gato_sonriendo:",
    ":grinning_cat_with_smiling_eyes:": ":gato_sonriendo_con_ojos_sonrientes:",
    ":cat_with_tears_of_joy:": ":gato_con_lágrimas_de_alegría:",
    ":smiling_cat_with_heart_eyes:": ":gato_sonriendo_con_ojos_de_corazón:",
    ":cat_with_wry_smile:": ":gato_con_sonrisa_irónica:",
    ":kissing_cat:": ":gato_besando:",
    ":weary_cat:": ":gato_cansado:",
    ":crying_cat:": ":gato_llorando:",
    ":pouting_cat:": ":gato_fruncido:",
    ":thumbsup:": ":pulgar_arriba:",
    "thumbs_up_medium_skin_tone:": ":pulgar_arriba:",
    ":thumbs_up:": ":pulgar_arriba:",
    "heart_hands_medium_skin_tone": ":manos formando un corazón:",
    "clapping_hands_light_skin_tone": ":manos clamando:"


# Puedes seguir agregando más traducciones según sea necesario
}

# Función para convertir emoticones a texto y luego traducirlos al español
def convertir_emoticones(texto):
    if isinstance(texto, str):  # Verifica si el input es una cadena
        texto_emoji = emoji.demojize(texto, delimiters=(":", ":"))
        for en, es in emoji_translations.items():
            texto_emoji = texto_emoji.replace(en, es)
        return texto_emoji
    else:
        return texto


In [12]:
# Aplicar la conversión de emoticones a la columna 'recibido'
data['recibido'] = data['recibido'].apply(convertir_emoticones)

In [13]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU está disponible y se usará.")
else:
  device = torch.device("cpu")
  print("GPU no está disponible, se usará la CPU.")

GPU está disponible y se usará.


In [14]:
# Aplicar la conversión de emoticones a la columna 'recibido'
data['recibido'] = data['recibido'].apply(convertir_emoticones)

# Convertir etiquetas de clase a representación numérica (suponiendo que son cadenas)
data['clase'] = data['clase'].astype('category')
data['clases_cat'] = data['clase'].cat.codes

# Filtrar las filas con etiquetas inválidas
data = data[data['clases_cat'] >= 0]

# Verificar que todas las etiquetas estén en el rango correcto
num_labels = len(data['clase'].unique())
print(f"Numero de etiquetas: {num_labels}")
print(f"Etiquetas unicas: {data['clases_cat'].unique()}")
assert data['clases_cat'].min() >= 0 and data['clases_cat'].max() < num_labels

Numero de etiquetas: 9
Etiquetas unicas: [0 8 2 3 7 6 5 1 4]


In [15]:
print(data.head())

                                             enviado  \
0   ¡Limpiar la casa, subir escaleras, trabajar e...   
1   ¡Limpiar la casa, subir escaleras, trabajar e...   
2   ¡Limpiar la casa, subir escaleras, trabajar e...   
3   ¡Limpiar la casa, subir escaleras, trabajar e...   
4   ¡Limpiar la casa, subir escaleras, trabajar e...   

                                            recibido                    clase  \
0   ¡Limpiar la casa, subir escaleras, trabajar e...  agradecimiento o saludo   
1   ¡Limpiar la casa, subir escaleras, trabajar e...  agradecimiento o saludo   
2   ¡Limpiar la casa, subir escaleras, trabajar e...  agradecimiento o saludo   
3   ¡Limpiar la casa, subir escaleras, trabajar e...  agradecimiento o saludo   
4   ¡Limpiar la casa, subir escaleras, trabajar e...                respuesta   

                                   recibido_original  clases_cat  
0      ::pulgar_arriba:::manos formando un corazón::           0  
1                                         

In [16]:
# Extraer la columna 'recibido'
text_data = data['recibido']

# Crear una instancia de CountVectorizer
vectorizer = CountVectorizer()

# Ajustar y transformar los datos de texto
X = vectorizer.fit_transform(text_data)

# Convertir la matriz de características a un DataFrame
X_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

# Mostrar la matriz de características
print(X_df)
print(X_df.iloc[:10, :10])  # Aquí mostramos las primeras 10 filas (comentarios de usuarios) y 10 columnas para una visualización más legible.

      00  000  00hs  04  05  06  08  0800  09  0km  ...  últimos  única  \
0      0    0     0   0   0   0   0     0   0    0  ...        0      0   
1      0    0     0   0   0   0   0     0   0    0  ...        0      0   
2      0    0     0   0   0   0   0     0   0    0  ...        0      0   
3      0    0     0   0   0   0   0     0   0    0  ...        0      0   
4      0    0     0   0   0   0   0     0   0    0  ...        0      0   
...   ..  ...   ...  ..  ..  ..  ..   ...  ..  ...  ...      ...    ...   
7002   0    0     0   0   0   0   0     0   0    0  ...        0      0   
7003   0    0     0   0   0   0   0     0   0    0  ...        0      0   
7004   0    0     0   0   0   0   0     0   0    0  ...        0      0   
7005   0    0     0   0   0   0   0     0   0    0  ...        0      0   
7006   0    0     0   0   0   0   0     0   0    0  ...        0      0   

      únicamente  único  útil  útiles  ďecime  ģracias  𝘚𝘪  𝘲𝘶𝘪𝘦𝘳𝘰  
0              0      0     0 

In [15]:
# Ahora vemos las 20 palabras mas utilzadas
word_counts = X.toarray().sum(axis=0)
words = vectorizer.get_feature_names_out()
word_freq = list(zip(words, word_counts))
word_freq_sorted = sorted(word_freq, key=lambda x: x[1], reverse=True)

In [16]:
top_words = word_freq_sorted[:20]
for word, freq in top_words:
    print(f'{word}: {freq}')


de: 7850
la: 5971
que: 5489
recibirlos: 4974
para: 4392
si: 4339
tu: 3869
no: 3595
con: 3548
el: 3482
quiero: 3441
en: 3088
te: 2927
un: 2887
diabetes: 2736
es: 2579
por: 2466
sí: 2401
los: 2370
mensajes: 2253


Al aplicar BoW perdemos el contexto del texto original y por otro lado queda sesgado hacia las palabras que ocurren muy frecuentemente pero no tienen mucho significado (ej: de, que, en, etc).

Aplicamos Tf-IDF para resaltar las palabras "exclusivas" o de mayor relevancia.

In [18]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(text_data)

In [19]:
feature_names = vectorizer.get_feature_names_out()
tfidf_df = pd.DataFrame(X.toarray(), columns=feature_names)
print(tfidf_df.head(10))  # Mostrar los primeros 10 documentos (textos) con sus TF-IDF

    00  000  00hs   04   05   06   08  0800   09  0km  ...  últimos  única  \
0  0.0  0.0   0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  ...      0.0    0.0   
1  0.0  0.0   0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  ...      0.0    0.0   
2  0.0  0.0   0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  ...      0.0    0.0   
3  0.0  0.0   0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  ...      0.0    0.0   
4  0.0  0.0   0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  ...      0.0    0.0   
5  0.0  0.0   0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  ...      0.0    0.0   
6  0.0  0.0   0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  ...      0.0    0.0   
7  0.0  0.0   0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  ...      0.0    0.0   
8  0.0  0.0   0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  ...      0.0    0.0   
9  0.0  0.0   0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  ...      0.0    0.0   

   únicamente  único  útil  útiles  ďecime  ģracias   𝘚𝘪  𝘲𝘶𝘪𝘦𝘳𝘰  
0         0.0    0.0   0.0     0.0     0.0      0.0  0.0     0.0  
1      

Los resultados obtenidos no son alentadores por la cantidad de palabras que componen el corpus (esto lo vemos en la cantidad de columnas - 3725 - que se indican al realizar la impresión del resultado de TfIdf).

Para solucionar esto y continuar con el análisis, aplicamos PCA para la reducción de dimensionalidad y de esta manera quedarnos con las 20 palabras mas representativas.

In [25]:
# Aplicar PCA para reducir la dimensionalidad
pca = PCA(n_components=20)
principal_components = pca.fit_transform(tfidf_df)

# Convertir los componentes principales a un DataFrame para visualización
pca_df = pd.DataFrame(principal_components)
# Mostrar los componentes principales
print(pca_df.head(20))


          0         1         2         3         4         5         6   \
0  -0.203299 -0.072521 -0.115060 -0.074778 -0.070741  0.061307 -0.056871   
1  -0.221151 -0.070641 -0.106192 -0.085411 -0.075303  0.079310 -0.066422   
2  -0.201350 -0.060334 -0.101906 -0.071218 -0.073368  0.038121 -0.056285   
3  -0.220495 -0.072719 -0.113115 -0.086644 -0.080516  0.087524 -0.070676   
4  -0.210459 -0.056996 -0.103555 -0.079345 -0.065430  0.050454 -0.047344   
5  -0.220637 -0.074656 -0.113903 -0.087226 -0.083382  0.088329 -0.072508   
6  -0.213513 -0.066896 -0.108994 -0.080657 -0.074879  0.071024 -0.064049   
7  -0.220637 -0.074656 -0.113903 -0.087226 -0.083382  0.088329 -0.072508   
8  -0.215557 -0.077440 -0.133419 -0.069974 -0.083584  0.098758 -0.028551   
9  -0.215306 -0.067856 -0.110266 -0.082286 -0.075826  0.075255 -0.065750   
10 -0.213225 -0.032063 -0.091468 -0.042048 -0.077325  0.105168 -0.062585   
11 -0.220495 -0.072719 -0.113115 -0.086644 -0.080516  0.087524 -0.070676   
12 -0.220340

In [22]:
# Identificar las palabras más importantes en los componentes principales
# Para esto, necesitamos obtener las cargas de los componentes
components = pca.components_

# Crear un DataFrame con las cargas de los componentes
components_df = pd.DataFrame(components, columns=feature_names)

In [29]:
print(components.shape)

(20, 3725)


In [23]:
# Para cada componente principal, mostrar las palabras con las mayores cargas absolutas
top_n = 20
for i in range(components.shape[0]):
    print(f"Componente Principal {i + 1}:")
    sorted_components = components_df.iloc[i].abs().sort_values(ascending=False)
    print(sorted_components.head(top_n))

Componente Principal 1:
recibirlos    0.234782
semanal       0.209445
recibirás     0.209431
bienvenida    0.205051
damos         0.204895
enviando      0.204895
cuidarte      0.203943
baja          0.203815
dejar         0.203336
recibir       0.202853
estos         0.202160
mejorar       0.201275
mensaje       0.192026
consejos      0.189823
número        0.187383
mensajes      0.187010
este          0.186013
quiero        0.185721
podés         0.184075
salud         0.158113
Name: 0, dtype: float64
Componente Principal 2:
recibirlos     0.467766
respondé       0.311264
querés         0.288497
quiero         0.247696
por            0.246156
no             0.210004
si             0.180091
enviarte       0.155632
aprobación     0.155632
semanales      0.155632
necesitamos    0.155632
whatsapp       0.155211
favor          0.154329
saludables     0.147799
sí             0.126163
semanal        0.109931
recibirás      0.109869
cuidarte       0.109083
mejorar        0.108551
bienvenida  

El resultado anterior lo muestra abierto por cada componente principal procesado (línea de comentario del dataframe), en nuestro caso indicamos las 20 primeras y por ese motivo solo vemos 20 componentes procesados.
Para cada línea muestra las palabras con su respectivo peso/carga

Ahora sumamos las puntuaciones de TF-IDF para cada palabra de todo el corpus y obtener así una visión general de las 50 palabras con mayor puntaje.

In [31]:
# Ahora sumamos las puntuaciones de TF-IDF para cada palabra de todo el corpus y obtener así una visión general.
tfidf_sums = tfidf_df.sum(axis=0)

# Ordenar las palabras por sus puntuaciones TF-IDF totales
sorted_tfidf = tfidf_sums.sort_values(ascending=False)

# Mostrar las 50 palabras más importantes
top_50_words = sorted_tfidf.head(50)
print(top_50_words)

recibirlos    592.261898
de            522.628671
la            504.668074
que           454.947522
quiero        421.291634
si            396.341298
no            362.502062
para          359.729047
tu            355.084512
con           340.352545
te            316.007959
sí            313.016422
un            312.101064
mensajes      300.725302
este          296.127679
el            295.061155
consejos      294.972481
mensaje       292.711098
salud         288.945775
diabetes      285.472955
por           283.604254
en            275.039083
número        266.899215
es            266.140012
podés         262.560956
querés        256.211785
mejorar       255.715804
baja          255.384165
recibir       255.168884
cuidarte      252.955406
dejar         252.314985
semanal       250.092845
recibirás     249.881672
respondé      249.507360
estos         249.474842
bienvenida    249.298955
enviando      249.096316
damos         249.096316
los           243.736829
se            234.089650
